<a href="https://colab.research.google.com/github/nimajr/MachineLearning2023/blob/main/HW2/Q4/HW2Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Q4-1&2**


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

# بارگذاری دیتاست
data = fetch_california_housing()
X, y = data.data, data.target
y = y.reshape(-1, 1)  # تغییر شکل برای سازگاری با مدل

# نرمال‌سازی داده‌ها
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# تقسیم داده‌ها به آموزش و آزمون
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# تبدیل به تنسور
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

# تعریف لایه RBF
class RBF_Layer(nn.Module):
    def __init__(self, input_dim, num_centers):
        super(RBF_Layer, self).__init__()
        self.centers = nn.Parameter(torch.randn(num_centers, input_dim))  # مراکز
        self.beta = nn.Parameter(torch.ones(1))  # پارامتر بتا

    def forward(self, x):
        # محاسبه فاصله اقلیدسی و تبدیل آن به تابع گاوسی
        diff = x.unsqueeze(1) - self.centers.unsqueeze(0)
        dist_sq = torch.sum(diff ** 2, dim=-1)
        return torch.exp(-self.beta * dist_sq)

# تعریف مدل کلی
class RBF_Network(nn.Module):
    def __init__(self, input_dim, num_centers, output_dim):
        super(RBF_Network, self).__init__()
        self.rbf_layer = RBF_Layer(input_dim, num_centers)
        self.linear_layer = nn.Linear(num_centers, output_dim)

    def forward(self, x):
        rbf_out = self.rbf_layer(x)
        output = self.linear_layer(rbf_out)
        return output

# تعریف مدل
input_dim = X_train.shape[1]
num_centers = 10  # تعداد نودهای لایه RBF
output_dim = 1
model = RBF_Network(input_dim, num_centers, output_dim)

# تنظیمات آموزش
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# آموزش مدل
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# ارزیابی مدل
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)
    print(f"Test Loss: {test_loss.item():.4f}")


Epoch [10/100], Loss: 1.0106
Epoch [20/100], Loss: 0.9975
Epoch [30/100], Loss: 0.9841
Epoch [40/100], Loss: 0.9420
Epoch [50/100], Loss: 0.8490
Epoch [60/100], Loss: 0.7198
Epoch [70/100], Loss: 0.5758
Epoch [80/100], Loss: 0.5162
Epoch [90/100], Loss: 0.4785
Epoch [100/100], Loss: 0.4503
Test Loss: 0.4528


# **Q4-2.4**

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing()
X, y = data.data, data.target
y = y.reshape(-1, 1)  # تغییر شکل برای سازگاری با مدل

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

class Dense_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Dense_Network, self).__init__()
        self.hidden_layer = nn.Linear(input_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.activation = nn.ReLU()

    def forward(self, x):
        hidden_output = self.activation(self.hidden_layer(x))
        output = self.output_layer(hidden_output)
        return output

# تنظیمات مدل
input_dim = X_train.shape[1]  # تعداد ویژگی‌ها
hidden_dim = 64  # تعداد نودهای لایه مخفی
output_dim = 1   # خروجی (قیمت خانه‌ها)

model = Dense_Network(input_dim, hidden_dim, output_dim)

# تنظیمات آموزش
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# آموزش مدل
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# ارزیابی مدل
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)
    print(f"Test Loss: {test_loss.item():.4f}")


Epoch [10/100], Loss: 0.4769
Epoch [20/100], Loss: 0.3833
Epoch [30/100], Loss: 0.3452
Epoch [40/100], Loss: 0.3259
Epoch [50/100], Loss: 0.3094
Epoch [60/100], Loss: 0.2969
Epoch [70/100], Loss: 0.2859
Epoch [80/100], Loss: 0.2767
Epoch [90/100], Loss: 0.2692
Epoch [100/100], Loss: 0.2629
Test Loss: 0.2707
